In [ ]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
import tensorflow
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [ ]:

english_stops = stopwords.words('english')


In [ ]:
def load_dataset():
    df = pd.read_csv('/content/train.csv',header=None, nrows=100000)
    df.columns = ['Polarity', 'Title', 'Review']
    df = df[['Polarity', 'Review']].reset_index(drop=True)
    x_data = df['Review']       # Reviews/Input
    y_data = df['Polarity']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # #ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace(1, 0)
    y_data = y_data.replace(2, 1)


    return x_data, y_data

x_data, y_data = load_dataset()

print('Review')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Review
0        [this, sound, track, beautiful, it, paints, se...
1        [i, reading, lot, reviews, saying, best, game,...
2        [this, soundtrack, favorite, music, time, hand...
3        [i, truly, like, soundtrack, i, enjoy, video, ...
4        [if, played, game, know, divine, music, every,...
                               ...                        
99995    [good, excelent, fantastic, wonderful, muy, bu...
99996    [these, are, the, real, adventures, of, the, r...
99997    [sylvain, beauregard, widely, known, die, hard...
99998    [i, think, i, know, person, wrote, book, i, ce...
99999    [this, call, overkill, how, many, books, need,...
Name: Review, Length: 100000, dtype: object 

Sentiment
0        1
1        1
2        1
3        1
4        1
        ..
99995    1
99996    0
99997    0
99998    0
99999    0
Name: Polarity, Length: 100000, dtype: int64


In [8]:
print(y_data.value_counts());

1    51267
0    48733
Name: Polarity, dtype: int64


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
35539    [while, cool, donal, trump, cologne, bathroom,...
51509    [it, twenty, years, since, i, saw, original, c...
77703    [in, real, food, nigel, slater, manages, invok...
52106    [royalty, changing, destiny, stars, new, world...
61906    [the, first, chapters, talk, xml, last, chapte...
                               ...                        
50459    [this, movie, one, best, i, seen, jami, gertz,...
60148    [i, paid, attention, cd, disappointed, it, mus...
85797    [wrong, adapter, satisfied, order, was, suppos...
5207     [heroes, civilization, cottler, jaffe, attempt...
65750    [socket, sizes, correct, causing, stripped, bo...
Name: Review, Length: 80000, dtype: object 

77990    [it, worked, nicely, months, literally, holds,...
68133    [before, i, start, i, would, like, say, i, bel...
14434    [shame, pieces, crap, feeding, well, oiled, ma...
9468     [i, really, liked, book, it, classic, rinaldi,...
8387     [the, one, hundred, years, solitude, gabriel, ...
 

In [ ]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [ ]:
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[  357   434 29768 ...     0     0     0]
 [    6  2320    56 ...  1418    31  1418]
 [   78   110   982 ...   192   157  3704]
 ...
 [  245  1118  1455 ...     0     0     0]
 [ 2905  4577 90139 ...     0     0     0]
 [ 9966  2961  1057 ...   258   238  2896]] 

Encoded X Test
 [[   6  330 1504 ...    0    0    0]
 [1532    1  304 ...   95   39   29]
 [ 878  729  968 ... 3860 6053 1286]
 ...
 [   5  903  408 ...    0    0    0]
 [   2 3887  256 ...   25  120   14]
 [   2    3  305 ...    0    0    0]] 

Maximum review length:  44


In [ ]:
EMBED_DIM = 32
LSTM_OUT = 64
model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 44, 32)            2884512   
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2909409 (11.10 MB)
Trainable params: 2909409 (11.10 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size = 128, epochs = 3)

Epoch 1/3
625/625 [==============================] - 64s 98ms/step - loss: 0.4130 - accuracy: 0.8059 - val_loss: 0.3471 - val_accuracy: 0.8506
Epoch 2/3
625/625 [==============================] - 70s 112ms/step - loss: 0.2713 - accuracy: 0.8901 - val_loss: 0.3826 - val_accuracy: 0.8420
Epoch 3/3
625/625 [==============================] - 61s 98ms/step - loss: 0.2092 - accuracy: 0.9188 - val_loss: 0.3874 - val_accuracy: 0.8344


In [ ]:
y_pred =  (model.predict(x_test) > 0.7).astype("int32")

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)1*100))

625/625 [==============================] - 5s 8ms/step
Correct Prediction: 16351
Wrong Prediction: 3649
Accuracy: 81.755


In [ ]:
model.save('LSTM_AmazonReview_T1.keras')
loaded_model = load_model('LSTM_AmazonReview_T1.keras')


In [ ]:
review = str(input('Movie Review: '))

Movie Review:  It was okay.


In [ ]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  It was okay
Filtered:  ['it okay']


In [ ]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[  7 712   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0]]


In [ ]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 [==============================] - 1s 1s/step
[[0.5428232]]


In [ ]:
if result >= 0.5:
    print('positive')
else:
    print('negative')

positive
